Twee versie tabellen 

ANN
- (+) trailers
- (+) '_|'
- trailer bestaat uit 2 cijfers achter elkaar op voorwaarde dat je al een spatie hebt gezien op die regel, een verschil van 2
- zelfde geldt voor '_|' elke keer dar je dit verschijnsel tegenkomt is er een verschil van 2
i = 0
ann_ann = []
for line in file:
    lijn lezen tot na eerste spatie (versnummer en spatie)
    for character in line:
        i += 1
        if character in '[0-9]{2}|_\|': (regex inzetten: )
            ann_ann.append(i)

MiMi
- (-) trailers
- (+) hypens en plussen 
- bestaat ui 1 positie
i = 0
ann_mimi = []
for line in file:
    for character in line:
        i += 1
        if character in {+, -}:
            ann_mimi.append(i)
            

Aanpak
- Iteren over elke Psalm, haal ik karakters uit, sla de index van het karakter op
- 2 index lijsten maken: ann_ann_list, ann_mimi_list
- ann_ann samenvoegen met ann_mimi in position_dict. position_dict is per chapter, dan leegmaken
- vertalen van oud (ann) naar nieuw (mimi). 
- op alle posities waar er iets verspringt, in de nieuwe of oude file, heb je een entry in position_dict
- de verspringpunten zijn de keys; het aantal karakters dat het verschilt de values
- vb voor Psalm 1: index 7 : int(1) (h-isj), voor mimifile '-' +1 
                   index 14 : int(-1) asjer05, voor annfile '05' -2 
                   
- iaa mention dict vullen met aangepaste mention strings die indices van annotaties bevatten 'mention 1 5', begin index 1 en eind index 5 dus apart aanpassen
- mention vertaler functie: def translate_mention()
- binnen translate_mention(), functie: def translate_index() die begin en eind index aanpast, gebruik daar voor Reindex() uit declust.py

PS_A   = $(HOME)/Sites/brat/data/coref/Psalms/annotate
PS_B   = $(HOME)/github/cmerwich/participant-analysis/mimi

In [ ]:
__author__ = 'erwich/sikkel'

In [1]:
import getopt
import os, sys
from sys import argv, exit, stderr
import re
from pprint import pprint
from collections import OrderedDict
from operator import attrgetter

In [2]:
#pattern = re.compile('\&|[0-9]{2}|_\|')
# = re.compile('[0-9]{1,3}[\s]') # match verses

PATTERN = re.compile('[0-9]{2}|_\|')
TAB = '\t'
NL = '\n'

In [3]:
class Mention:
    def __init__(self, name, start=0, end=0, lex='', note='', file=0):
        self.name = name    # Identifier of the mention, e.g. T32
        self.start = start  # Start of the position in the file
        self.end = end      # End of the position in the file
        self.lex = lex      # Lexical information
        self.note = note    # AnnotatorNotes
        self.file = file    # Index of the file to which it is to be written

In [4]:
def error(*args, **kwargs):
    print(*args, file=stderr, **kwargs)
    exit(1)

In [5]:
def Usage():
    stderr.write('usage: translate -a input.ann -o output.ann old.txt new.txt\n')
    exit(1)

In [6]:
def GetOffsetsAnn(chapter_text):
    
    i = 0
    ann_txt_list = []
    with open(chapter_text) as fh:
        for (ln, line) in enumerate(fh):
            match = PATTERN.finditer(line)
            for character in line:
                i += 1
                print('c: ', i, character)
                for m in match:
                    # filter out psalms title on line 0, 
                    # and (0, 2) for verse numbers with two digits or more
                    if ln != 0 and m.span() != (0, 2):
                        #ann_txt_list.append(f'{i+m.start()} {i+m.start()+1}') #{m.end()}
                        ann_txt_list.append((i+m.start(), m.group()))                                    
    return ann_txt_list

In [7]:
def GetOffsetsMimi(chapter_text):
    
    i = 0
    mimi_txt_list = []
    
    with open(chapter_text) as fh:
        for (ln, line) in enumerate(fh):
            firstChar = line[0]
            for character in line:
                i += 1
                print('m: ', i, character)
                if character in {'+', '-'}:
                    mimi_txt_list.append((i-1, character))
                    
    return mimi_txt_list         

In [8]:
def elision(new, i):
    return new[i][1] == '+' or i > 0 and new[i-1][0] + 1 == new[i][0] and new[i-1][1] == '-'


def Merge(old, new):
    offset = 0
    jumps_list = [(0, 0)]
    
    i_old = 0
    i_new = 0
    in_old = i_old < len(old)
    in_new = i_new < len(new)
    
    while in_old or in_new:
        # < or <= ? 
        # is het -offset, of + offset
        if not in_new or (in_old and old[i_old][0] - offset < new[i_new][0]):
            # add old 
            x = old[i_old][0] - offset
            offset += 2
            i_old += 1 
            in_old = i_old < len(old)
        else: 
            # add new
            x = new[i_new][0] + elision(new, i_new)
            offset -= 1
            i_new += 1
            in_new = i_new < len(new)
        jumps_list.append((x, offset))
            
    return jumps_list

In [9]:
def find_index(jumps_list, coor):
    i = 0
    while i + 1 < len(jumps_list) and jumps_list[i+1][0] <= coor:
        i += 1
    return i

def Offset(coor, jumps_list):
    i = find_index(jumps_list, coor)
    return jumps_list[i][1]

def TranslateMentions(mentions, jumps_list):
    i = 0
    for m in mentions:
        print(m.end)
        i += 1
        #m.start += jumps_list[i][1]#[1 if i == coor else 2] #m.start += jumps_list[i][1 if i == coor else 2]
        #m.end += jumps_list[i][1] #[2]
        
        m.start += Offset(m.start, jumps_list)
        m.end += Offset(m.end, jumps_list)

In [10]:
def Parse(ann_file):
    '''
    Parses a plain text annotation file per Bible Book chapter 
    (e.g. Psalms 001) that has been annotated for coreference information.
    Returns a list of lists of coref-mention objects and 
    a list of mention objects. 
    '''
    
    errors = 0
    t2mDict = {}
    Mentions = []
    Corefs = []
    MentionIndexDict = {}
    mentionNote = {}
    dataPartsList = []

    firstChars = {'T', '#', '*'}
    cClass = 0

    with open(ann_file) as fh:
        for (i, line) in enumerate(fh):
            epos = f'{i + 1} '
            line = line.rstrip('\n')
            firstChar = line[0]

            if firstChar not in firstChars:
                error(f'{epos}Unrecognized line "{line}"')
                errors +=1
                continue

            numFields = 2 if firstChar =='*' else 3
            parts = line.split('\t')

            if len(parts) != numFields:
                error(f'{epos}line does not have exactly {numFields} parts: "{line}"')
                errors += 1
                continue

            if firstChar == 'T':
                (tPart, mentionStr, aWord) = parts
                mParts = mentionStr.split()
                if len(mParts) != 3:
                    error(f'{epos}T-line mention does not have exactly 3 parts: "{line}"')
                    errors += 1
                    continue
                (mm, aStart, aEnd) = mParts
                start = int(aStart)
                end = int(aEnd)
                t2mDict[tPart] = mentionStr
                m = Mention(tPart, start, end, aWord)
                MentionIndexDict[tPart] = len(Mentions)
                Mentions.append(m)
                
            elif firstChar == '*':
                corefSets = set()
                (char, data) = parts
                dataParts = data.split()
                if len(dataParts) <= 1 or dataParts[0] != 'Coreference':
                    error(f'{epos}*-line spec does not have the right parts: "{line}"')
                    errors += 1
                    continue
                cClass += 1
                dataPartsList.append(dataParts)
            
            elif firstChar == '#':
                (code, spec, note) = parts
                sParts = spec.split()
                if len(sParts) != 2:
                    error(f'{epos}#-line spec does not have exactly 2 parts: "{line}"')
                    errors += 1
                    continue
                tPart = sParts[1]
                if tPart in MentionIndexDict:
                    Mentions[MentionIndexDict[tPart]].note = note
                else:
                    error(f'Annotatornote "{note}": {tPart}: Mention not found')
                    sys.exit(1)
                
                mentionNote.setdefault(tPart, set()).add(note)
                   
        for l in dataPartsList:
            corefSets = set() # for error analysis 
            corefLists = []
            for tPart in l[1:]:
                if tPart in corefSets:
                    error(f'{epos}*-"{tPart} occurs in multiple classes "{corefSets[tPart]}" in "{line}"')
                    errors += 1
                    continue
                corefSets.add(t2mDict[tPart]) # for error analysis 
                if tPart in MentionIndexDict:
                    corefLists.append(Mentions[MentionIndexDict[tPart]])
                else:
                    error(f'Coref: {tPart}: Mention not found')
                    sys.exit(1)
            Corefs.append(corefLists) # list of lists, one list is one coreference class. 
            
    if errors:
        error(f'There are {errors} errors in annotation file(s)')
    
    return Mentions, Corefs

In [11]:
def OpenAnn(path):
    '''
    Opens an annotation file with extension `.ann' for each chapter
    in current folder. 
    '''
    
    ann_file = open(path, 'w')
    
    return ann_file

In [12]:
def WriteMentions(mentions, ann_file):
    '''
    Writes the mentions and annotatornotes on mentions to the corresponding ann file. 
    '''
    
    i = 0
    sorted_mentions = sorted(mentions, key=attrgetter('start'))
    for m in sorted_mentions:
        ann_file.write(f'{m.name}{TAB}Mention {str(m.start)} {str(m.end)}{TAB}{m.lex}{NL}')
        
        if m.note != '':
            i += 1
            ann_file.write(f'#{str(i)}{TAB}AnnotatorNotes {m.name}{TAB}{m.note}{NL}')

In [13]:
def WriteCorefs(corefs, ann_file):
    '''
    Writes a coref class with mentions to .ann file object.
    '''
    
    for c in corefs:
        ann_file.write(f'*{TAB}Coreference')
        sorted_corefs = sorted(c, key=attrgetter('start'))
        for m in sorted_corefs:
            ann_file.write(f' {m.name}') 
        ann_file.write('\n')

In [14]:
def Close(file): 
    '''
    Close ann file object.
    '''
    
    file.close()

In [15]:
def translate(new_ann, out, texts):
    # new_ann = mimi file.ann
    # out = mimi translated to chris file.ann
    # texts = list[old_file.txt, mimi_file.txt]
    
    old = GetOffsetsAnn(texts[0])
    new = GetOffsetsMimi(texts[1])
    jumps_list = Merge(old, new)
    print('old: ', old, '\n')
    print('new: ', new, '\n')
    pprint(jumps_list)
    Mentions, Corefs = Parse(new_ann)
    TranslateMentions(Mentions, jumps_list)
    mimi_ann_file = OpenAnn(out)
    WriteMentions(Mentions, mimi_ann_file)
    WriteCorefs(Corefs, mimi_ann_file)
    Close(mimi_ann_file)
    

In [16]:
#mimi ann file: T35	Mention 242 250	G>- JWNJM
translate('Psalms_123-mimi.ann', 'Psalms_123_n.ann', ['Psalms_123-chris.txt', 'Psalms_123-mimi.txt'])

c:  1 P
c:  2 s
c:  3 a
c:  4 l
c:  5 m
c:  6 s
c:  7 _
c:  8 1
c:  9 2
c:  10 3
c:  11 

c:  12 1
c:  13  
c:  14 C
c:  15 J
c:  16 R
c:  17  
c:  18 H
c:  19 M
c:  20 <
c:  21 L
c:  22 W
c:  23 T
c:  24  
c:  25 >
c:  26 L
c:  27 J
c:  28 K
c:  29  
c:  30 N
c:  31 F
c:  32 >
c:  33 T
c:  34 J
c:  35  
c:  36 >
c:  37 T
c:  38 &
c:  39 <
c:  40 J
c:  41 N
c:  42 J
c:  43  
c:  44 H
c:  45 J
c:  46 C
c:  47 B
c:  48 J
c:  49  
c:  50 B
c:  51 C
c:  52 M
c:  53 J
c:  54 M
c:  55 0
c:  56 0
c:  57  
c:  58 

c:  59 2
c:  60  
c:  61 H
c:  62 N
c:  63 H
c:  64  
c:  65 K
c:  66 <
c:  67 J
c:  68 N
c:  69 J
c:  70  
c:  71 <
c:  72 B
c:  73 D
c:  74 J
c:  75 M
c:  76  
c:  77 >
c:  78 L
c:  79 &
c:  80 J
c:  81 D
c:  82  
c:  83 >
c:  84 D
c:  85 W
c:  86 N
c:  87 J
c:  88 H
c:  89 M
c:  90  
c:  91 K
c:  92 <
c:  93 J
c:  94 N
c:  95 J
c:  96  
c:  97 C
c:  98 P
c:  99 X
c:  100 H
c:  101  
c:  102 >
c:  103 L
c:  104 &
c:  105 J
c:  106 D
c:  107  
c:  108 G
c:  109 B
c:  110 R
c:  111 